<a href="https://colab.research.google.com/github/jeffreylowzg/LLM_homework6/blob/jeffrey-commits/data_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install -U "huggingface_hub[cli]"
!pip install torch transformers[torch] numpy tqdm datasets peft accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Download dataset and saves 5%

In [2]:
from datasets import load_dataset
import pandas as pd
import os

# Load the dataset from Hugging Face
dataset = load_dataset("dmitva/human_ai_generated_text", split="train")

# Calculate 5% of the dataset size
sample_size = int(0.05 * len(dataset))

# Sample 5% of the data
sampled_dataset = dataset.shuffle(seed=42).select(range(sample_size))

# Convert to pandas DataFrame for easier handling
df = pd.DataFrame(sampled_dataset)

# Ensure the 'data' directory exists
os.makedirs("data", exist_ok=True)

# Save to a CSV file in the 'data' folder
df.to_csv("data/sample_5_percent.csv", index=False)

print("5% of the dataset has been saved to 'data/sample_5_percent.csv'")


/root/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


5% of the dataset has been saved to 'data/sample_5_percent.csv'


Read saved data and split into labels 0 (for human) and 1 (for ai)

In [3]:
import json

# Read the sampled CSV file
df = pd.read_csv("data/sample_5_percent.csv")

# Initialize an empty list to hold the new records
data = []

# Process each row to create two entries: one for human text, one for AI text
for _, row in df.iterrows():
    # Append the human text with label 0
    data.append({
        "text": row["human_text"],
        "instructions": row["instructions"],
        "label": 0
    })

    # Append the AI text with label 1
    data.append({
        "text": row["ai_text"],
        "instructions": row["instructions"],
        "label": 1
    })

# Save the processed data to a JSON file
outfile = "data/sample_5_percent.jsonl"
with open(outfile, "w") as f:
    for d in data:
        json.dump(d, f)
        f.write("\n")

print(f"The dataset has been saved to {outfile} with the specified format.")

The dataset has been saved to data/sample_5_percent.jsonl with the specified format.


check if jsonl file is formatted correctly

In [4]:
import json

data_path = "data/sample_5_percent.jsonl"

with open(data_path, 'r') as f:
    for i, line in enumerate(f, 1):
        try:
            json.loads(line)
        except json.JSONDecodeError as e:
            print(f"JSON decode error on line {i}: {e}")


In [5]:
!mkdir -p models/pythia-160m
!huggingface-cli download EleutherAI/pythia-160m --local-dir ./models/pythia-160m
!cd ../..

Fetching 8 files:   0%|                                   | 0/8 [00:00<?, ?it/s]Downloading '.gitattributes' to 'models/pythia-160m/.cache/huggingface/download/.gitattributes.c7d9f3332a950355d5a77d85000f05e6f45435ea.incomplete'

.gitattributes: 100%|██████████████████████| 1.48k/1.48k [00:00<00:00, 3.00MB/s]
Download complete. Moving file to models/pythia-160m/.gitattributes
Fetching 8 files:  12%|███▍                       | 1/8 [00:00<00:00,  7.38it/s]Downloading 'README.md' to 'models/pythia-160m/.cache/huggingface/download/README.md.2e8b2b93bf534833600f862c90c0b53fc6f76a46.incomplete'

special_tokens_map.json: 100%|████████████████| 99.0/99.0 [00:00<00:00, 991kB/s]
Download complete. Moving file to models/pythia-160m/special_tokens_map.json

tokenizer.json:   0%|                               | 0.00/2.11M [00:00<?, ?B/s]

README.md: 100%|███████████████████████████| 13.6k/13.6k [00:00<00:00, 28.1MB/s]
Download complete. Moving file to models/pythia-160m/README.md


model.safetensor

In [6]:
!wandb login 7077b7416aa6d8dd6e87ab0b9150b82abed30bd1

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


original training script. can delete (not used)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

# Specify the local directory where the model was downloaded
model_path = "./models/pythia-160m"

# Load the tokenizer and model for sequence classification
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2)  # Two labels for binary classification

# Add padding token if it doesn't exist and set it as the pad token
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))  # Resize model embeddings to match the new pad token

# Explicitly set pad_token_id in model configuration
model.config.pad_token_id = tokenizer.pad_token_id

# Load the dataset
data_path = "data/sample_5_percent.jsonl"
dataset = load_dataset("json", data_files=data_path)

# Preprocessing function for tokenization and label mapping
def preprocess_function(examples):
    # Tokenize the text
    inputs = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)
    inputs["labels"] = examples["label"]  # Use label for classification
    return inputs

# Tokenize the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Split the dataset for training and evaluation
split_datasets = tokenized_dataset["train"].train_test_split(test_size=0.2)
train_dataset = split_datasets["train"]
eval_dataset = split_datasets["test"]

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./models/pythia-160m-finetuned-classifier",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    save_strategy="epoch",  # Set save_strategy to epoch
    evaluation_strategy="epoch",
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=100,
    load_best_model_at_end=True,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Save the final model
model.save_pretrained("./models/pythia-160m-finetuned-classifier")
tokenizer.save_pretrained("./models/pythia-160m-finetuned-classifier")

print("Model fine-tuning completed and saved to './models/pythia-160m-finetuned-classifier'")


lora + freeze first few layers (tested working)

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from peft import LoraConfig, get_peft_model

# Specify the local directory where the model was downloaded
model_path = "./models/pythia-160m"

# Load the tokenizer and model for sequence classification
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2)  # Binary classification

# Add padding token if it doesn't exist and set it as the pad token
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))  # Resize model embeddings to match the new pad token

# Explicitly set pad_token_id in model configuration
model.config.pad_token_id = tokenizer.pad_token_id

# LoRA Configuration
lora_config = LoraConfig(
    task_type="SEQ_CLS",   # Sequence classification
    inference_mode=False,
    r=16,                  # LoRA rank
    lora_alpha=32,         # Scaling factor
    lora_dropout=0.1       # Regularization
)

# Wrap the model with LoRA
model = get_peft_model(model, lora_config)

# Freeze the first few layers of GPT-NeoX
num_layers_to_freeze = 6  # Adjust based on model depth and dataset size

# For GPT-NeoX, transformer layers are in model.base_model.gpt_neox.layers
for layer in model.base_model.gpt_neox.layers[:num_layers_to_freeze]:
    for param in layer.parameters():
        param.requires_grad = False

# Always ensure the classification head and LoRA layers are trainable
model.print_trainable_parameters()  # Check trainable parameters

# Load the dataset
data_path = "data/sample_5_percent.jsonl"
dataset = load_dataset("json", data_files=data_path)

# Preprocessing function for tokenization and label mapping
def preprocess_function(examples):
    # Tokenize the text
    inputs = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)
    inputs["labels"] = examples["label"]  # Use label for classification
    return inputs

# Tokenize the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Split the dataset for training and evaluation
split_datasets = tokenized_dataset["train"].train_test_split(test_size=0.2)
train_dataset = split_datasets["train"]
eval_dataset = split_datasets["test"]

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./models/pythia-160m-finetuned-classifier-lora",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    save_strategy="epoch",     # Save the model at the end of each epoch
    evaluation_strategy="epoch",
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=100,
    load_best_model_at_end=True,
    learning_rate=1e-4,        # Adjusted for PEFT
    fp16=True,                 # Enable mixed precision training if supported
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Save the final model
model.save_pretrained("./models/pythia-160m-finetuned-classifier-lora")
tokenizer.save_pretrained("./models/pythia-160m-finetuned-classifier-lora")

print("Model fine-tuning completed and saved to './models/pythia-160m-finetuned-classifier-lora'")


/root/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of GPTNeoXForSequenceClassification were not initialized from the model checkpoint at ./models/pythia-160m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 296,448 || all params: 124,262,400 || trainable%: 0.2386


/root/miniconda3/lib/python3.12/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_9856/4112543343.py:81: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


[2024-11-18 22:30:32,160] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/root/miniconda3/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/root/miniconda3/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  On Ampere and higher architectures please use CUDA 11+
 [WARNING]  On Ampere and higher architectures please use CUDA 11+
 [WARNING]  On Ampere and higher architectures please use CUDA 11+
 [WARNING]  On Ampere and higher architectures please use CUDA 11+
 [WARNING]  On Ampere and higher architectures please use CUDA 11+
 [WARNING]  On Ampere and higher architectures please use CUDA 11+


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: jzlow (jzlow-carnegie-mellon-university). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.011300,0.038874
2,0.012500,0.011636
3,0.008200,0.008856


/root/miniconda3/lib/python3.12/site-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/root/miniconda3/lib/python3.12/site-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/root/miniconda3/lib/python3.12/site-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/root/miniconda3/lib/python3.12/site-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/root/miniconda3/lib/python3.12/site-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has

Model fine-tuning completed and saved to './models/pythia-160m-finetuned-classifier-lora'


train + evaluate

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from sklearn.metrics import accuracy_score
import numpy as np

# Specify the local directory where the model was downloaded
model_path = "./models/pythia-160m"

# Load the tokenizer and model for sequence classification
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2)  # Binary classification

# Add padding token if it doesn't exist and set it as the pad token
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))  # Resize model embeddings to match the new pad token

# Explicitly set pad_token_id in model configuration
model.config.pad_token_id = tokenizer.pad_token_id

# LoRA Configuration
lora_config = LoraConfig(
    task_type="SEQ_CLS",   # Sequence classification
    inference_mode=False,
    r=16,                  # LoRA rank
    lora_alpha=32,         # Scaling factor
    lora_dropout=0.1       # Regularization
)

# Wrap the model with LoRA
model = get_peft_model(model, lora_config)

# Freeze the first few layers of GPT-NeoX
num_layers_to_freeze = 6  # Adjust based on model depth and dataset size

# For GPT-NeoX, transformer layers are in model.base_model.gpt_neox.layers
for layer in model.base_model.gpt_neox.layers[:num_layers_to_freeze]:
    for param in layer.parameters():
        param.requires_grad = False

# Always ensure the classification head and LoRA layers are trainable
model.print_trainable_parameters()  # Check trainable parameters

# Load the dataset
data_path = "data/sample_5_percent.jsonl"
dataset = load_dataset("json", data_files=data_path)

# Preprocessing function for tokenization and label mapping
def preprocess_function(examples):
    # Tokenize the text
    inputs = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)
    inputs["labels"] = examples["label"]  # Use label for classification
    return inputs

# Tokenize the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Split the dataset for training and evaluation
split_datasets = tokenized_dataset["train"].train_test_split(test_size=0.2)
train_dataset = split_datasets["train"]
eval_dataset = split_datasets["test"]

# Define a function to compute accuracy
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)  # Take the highest probability class
    accuracy = accuracy_score(labels, predictions)
    return {"accuracy": accuracy}

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./models/pythia-160m-finetuned-classifier-lora",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    save_strategy="epoch",     # Save the model at the end of each epoch
    evaluation_strategy="epoch",
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=100,
    load_best_model_at_end=True,
    learning_rate=1e-4,        # Adjusted for PEFT
    fp16=True,                 # Enable mixed precision training if supported
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,  # Add this line
)

# Train the model
trainer.train()

# Save the final model
model.save_pretrained("./models/pythia-160m-finetuned-classifier-lora")
tokenizer.save_pretrained("./models/pythia-160m-finetuned-classifier-lora")

print("Model fine-tuning completed and saved to './models/pythia-160m-finetuned-classifier-lora'")

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation Results: {eval_results}")


Some weights of GPTNeoXForSequenceClassification were not initialized from the model checkpoint at ./models/pythia-160m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 296,448 || all params: 124,262,400 || trainable%: 0.2386


/root/miniconda3/lib/python3.12/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_9856/2074539927.py:90: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.013500,0.011972,0.998050
2,0.005300,0.012388,0.998250
3,0.001400,0.005379,0.999050


/root/miniconda3/lib/python3.12/site-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/root/miniconda3/lib/python3.12/site-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/root/miniconda3/lib/python3.12/site-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/root/miniconda3/lib/python3.12/site-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/root/miniconda3/lib/python3.12/site-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has

Model fine-tuning completed and saved to './models/pythia-160m-finetuned-classifier-lora'


Evaluation Results: {'eval_loss': 0.0053788283839821815, 'eval_accuracy': 0.99905, 'eval_runtime': 57.6947, 'eval_samples_per_second': 346.652, 'eval_steps_per_second': 43.332, 'epoch': 3.0}
